# Contact Tracing

**here we implimente devices and contact or meeting as i call it**

## UTILS

**Some utilities functions**

In [1]:
import random
import string

def get_random_alphaNumeric_string(stringLength=8):
    lettersAndDigits = string.ascii_letters + string.digits
    return ''.join((random.choice(lettersAndDigits) for i in range(stringLength)))

len(get_random_alphaNumeric_string(32))

32

In [2]:
import hashlib
import hmac
from math import ceil

hash_len = 32
def hmac_sha256(key, data):
    return hmac.new(key, data, hashlib.sha256).digest()

def hkdf(length: int, ikm, salt: bytes = b"", info: bytes = b"") -> bytes:
    if len(salt) == 0:
        salt = bytes([0]*hash_len)
    prk = hmac_sha256(salt, ikm)
    t = b""
    okm = b""
    for i in range(ceil(length / hash_len)):
        t = hmac_sha256(prk, t + info + bytes([1+i]))
        okm += t
    return okm[:length]

In [3]:
# MAKING KEYS
import datetime;
from datetime import date
import time





# REQUESTS TO SERVER
import requests
import json

import base64

#VIEWS
from termcolor import colored


#THREADING
import threading
import time

import ipyparallel

## CLASS TRACING KEY

In [4]:
class Tracingkey:
    
    def __init__(self):
        self.__key_value = None
        self.__makeTracingKey()
        
    def getKeyValue(self):
        return self.__key_value
    
    def __makeTracingKey(self, length=32):
            self.__key_value = get_random_alphaNumeric_string(length)

## CLASS ROPLLING PROXIMITY IDENTIFIER

In [5]:
class RollingProximityKey:
    def __init__(self, proximiy_id, ttl=10):
        self.__id = proximiy_id
        self.time_to_live  = ttl # 10 minute
        
       
        
    def getKeyValue(self):
        return self.__id
    
    def getTTL(self):
        return self.time_to_live
        

## CLASS DAILY TRACING KEY

In [6]:
class DailyTracingKey:
    def __init__(self, day_number, tracing_key):
        
        self.__key_value = None
        
        self.__number_day = day_number
        self.__encodeDailyTracingKey(tracing_key) #generate __daily_tracing_key
        
        self.valid_from = day_number
        self.valid_to = day_number+1
        self.__is_gen = False
    
    
    def isValidKey(self, tracing_key):
        gen_key = hkdf(length=16, ikm = bytes(tracing_key, 'utf-8'), info = bytes(self.__number_day) ) 
        
        if gen_key == self.__key_value:
            return True
        return False
    
    def getKeyValue(self):
        return self.__key_value
    
    def getDayNum(self):
        return self.__number_day
        
    
    def generateRollingProximityIdentifierKey(self, daily_key = None, 
                                            time_interval = None):
        
        seconds_since_start_of_day_number =  datetime.datetime.now().timestamp()% (60*60*24)
        self.__time_interval_number = int(seconds_since_start_of_day_number/(60*10)) 
        
        
        if daily_key == None:
            daily_key = self.__key_value
        if time_interval == None:
            time_interval = self.__time_interval_number  
        
        time_interval_number_encoded = str(time_interval).encode()
        print('Time interval encoded', time_interval_number_encoded)
        self.__proximity_id_compete  = hmac.new( daily_key, time_interval_number_encoded , 
                                                hashlib.sha256).digest()
        self.__proximity_id_16 = self.__proximity_id_compete[0:16]
        
        self.__proximity_id_16
        
        rolling_proximity = RollingProximityKey(self.__proximity_id_16)
        
        return rolling_proximity
    
    
    
    def __encodeDailyTracingKey(self, tracing_key):
        #print(self.__tracing_key)
        '''
        if self.__is_gen = False:
            self.__key_value = hkdf(length=16, ikm = bytes(tracing_key, 'utf-8'), info = bytes(self.__number_day) )        
            self.__is_gen =True
            return
        print("already generated")
        '''
        self.__key_value = hkdf(length=16, ikm = bytes(tracing_key, 'utf-8'), info = bytes(self.__number_day) )     
        #]              print here 
        
    
    
    def __generateRollingProximityIdentifierKey(self, infected_diagnostic_keys_tuple, interval):
        __possible_rolling_proxomity_keys_tuples = []
        
        (derived_day, diagnostic_key) = infected_diagnostic_keys_tuple
        for interval_val in range(interval):
            
            proximity_id_compete  = hmac.new(
                base64.b64decode(diagnostic_key) , 
                str(interval_val).encode()  , 
                hashlib.sha256).digest()

            proximity_id_16 = proximity_id_compete[0:16]

            __possible_rolling_proxomity_keys_tuples.append((interval_val, proximity_id_16, derived_day))
        return __possible_rolling_proxomity_keys_tuples
    
    
    def generateAllProximityRollingKey(self, infected_diagnostic_key_dict, nb = None):
        if nb is None:
            nb = len(infected_diagnostic_key_dict)
        nb = min(len(infected_diagnostic_key_dict), nb)    
        
        self.all_possible_rolling_proximity_keyes = []
        infected_diagnostic_keys_tuples = []
        
        for derived_day, infected_diag_key_list in list(infected_diagnostic_key_dict.items())[-nb:]:

            infected_diagnostic_keys_tuples.extend([(derived_day, diag_key) for diag_key in infected_diag_key_list])
            
        ([self.all_possible_rolling_proximity_keyes.extend(self.__generateRollingProximityIdentifierKey(tuple_key, 144)) for tuple_key in infected_diagnostic_keys_tuples])
        
        return self.all_possible_rolling_proximity_keyes 
        
    
    
    

## CLASS TRACING KEY STORE

In [7]:
class TracingKeyStore:
    def __init__(self, owner_name, device_id, update_interval=300):
        
        '''IDENTIFICATION OF THE DEVICE'''
        self.device_id = device_id
        self.owner_name = owner_name
        '''SELF INFECTION'''
        self.__get_infected = False
        
        '''TRACING KEY IDENTIFIER'''
        self.__device_tracing_key = Tracingkey() #Tracing key for the key stoore (device)
        
        '''UPDATE INTERVAL'''
        self.__update_interval = update_interval
       
        
        
        
        '''DAILY TRACING KEYS'''
        self.__day_number = None #current number day
        self.__daily_tracing_key_queue = None # Daily tracing key queue table
        self.__daily_tracing_queue_len=14
        self.daily_tracing_cpt = -1 # So we can call updateState function
        self.__daily_tracing_key_queue = [None for i in range(self.__daily_tracing_queue_len)]
        
        
        #generate __daily_tracing_key
        self.__current_daily_tracing_object = None
        '''END DAILY TRACING KEYS'''
        
        
        '''ROLLING PROXIMITY INDENTIFIER'''
        self.__time_interval_number = None
        self.__generated_rolling_proxy_table = []
        #get first rolling identifier the rolling
        self.__current_rolling_proxomity_identifier = None
        '''END ROLLING PROXIMITY INDENTIFIER'''
        
        
        '''VERY IMPORANT WE CALL UPDATE STATE ON THE NEWLLY CREATED OBJECT TO SET STATE ON A THREAD'''
        '''
        #Using threads
        
        thread = threading.Thread(target=self.updateState(), args=())
        thread.daemon = True                            # Daemonize thread
        thread.start() 
        '''
        
        self.updateState()
        '''
        SET THE STATE
        '''
        
      
        '''CONTACT LIST'''
        self.__contact_rolling_proxy_table = []
        self.contact_list = None # Dailly tracing key
        
        ## pour les objet de type rolling key
        self.__tab_rolling_proxy_ids = []
        
        
        ##
        self.__get_infected = False
        
        
        '''INFECTION AREA'''
        
        ##
        self.__infected_diagnostic_keys_dict = {}
        
        ###
        self.__tab_infected_diag_keys_tuples = []
        
        ###
        self.__infections_days = []
        
    def getOwnerName(self):
        return self.owner_name
    def getDeviceId(self):
        return self.device_id
    
    def getCurrentTracingKey(self):
        return self.__device_tracing_key.getKeyValue()
    
    
    
    def getDailyKeyQueueIndex(self, inc=0, add=0):
        if inc !=0:
            self.daily_tracing_cpt += inc
            return (self.daily_tracing_cpt % self.__daily_tracing_queue_len)+add
        
        return (self.daily_tracing_cpt % self.__daily_tracing_queue_len)+add
        
        
        
        
    def updateStateWithThread(self):
        
        while True:
            
            
            
            if self.__day_number != int(datetime.datetime.now().timestamp()/(60*60*24)):
                self.__day_number = int(datetime.datetime.now().timestamp()/(60*60*24))

                self.__current_daily_tracing_object = DailyTracingKey(self.__day_number, self.__device_tracing_key.getKeyValue())
                daily_key_index = self.getDailyKeyQueueIndex(inc=1)
                self.__daily_tracing_key_queue[daily_key_index]= self.__current_daily_tracing_object


            seconds_since_start_of_day_number =  datetime.datetime.now().timestamp()% (60*60*24)    
            if self.__time_interval_number != int(seconds_since_start_of_day_number/(60*10)):
                self.__time_interval_number = int(seconds_since_start_of_day_number/(60*10))

                self.__current_rolling_proxomity_identifier = self.__current_daily_tracing_object.generateRollingProximityIdentifierKey()
                self.__generated_rolling_proxy_table.append(self.__current_rolling_proxomity_identifier)
            
            # Do background update
            
            print('Update state for Key Store ', self.device_id , "Owner ", self.owner_name)

            time.sleep(self.__update_interval)
            
            
    def updateState(self):
        

        if self.__day_number != int(datetime.datetime.now().timestamp()/(60*60*24)):
            self.__day_number = int(datetime.datetime.now().timestamp()/(60*60*24))

            self.__current_daily_tracing_object = DailyTracingKey(self.__day_number, self.__device_tracing_key.getKeyValue())
            daily_key_index = self.getDailyKeyQueueIndex(inc=1)
            self.__daily_tracing_key_queue[daily_key_index]= self.__current_daily_tracing_object


        seconds_since_start_of_day_number =  datetime.datetime.now().timestamp()% (60*60*24)    
        if self.__time_interval_number != int(seconds_since_start_of_day_number/(60*10)):
            self.__time_interval_number = int(seconds_since_start_of_day_number/(60*10))

            self.__current_rolling_proxomity_identifier = self.__current_daily_tracing_object.generateRollingProximityIdentifierKey()
            self.__generated_rolling_proxy_table.append(self.__current_rolling_proxomity_identifier)
        
        #update diagnostic keys
        #self.getInfectedDiagnosticKeys()
        # Do background update

        print('Update state for Key Store ', self.device_id , "Owner ", self.owner_name)


    
    def getCurrentDayNumber():
        '''Update the state, then get the current daily object'''
        self.updateState()
        return self.__day_number
    
    def getCurrentDailyTracingObject(self):
        '''Update the state, then get the current daily object'''
        self.updateState()
        
        return self.__current_daily_tracing_object

    def getCurrentDailyTracingKey(self):
        '''Update the state, then get the current daily key'''
        self.updateState()
        
        return self.__current_daily_tracing_object.getKeyValue()
        #return self.getCurrentDailyTracingObject().getKeyValue()
        
        
    
    def getCurrentRollingProximityIdentifier(self):
        '''Update the state, then get the current rolling proximity key'''
        self.updateState()
        
        return self.__current_rolling_proxomity_identifier
    
    
    def saveContact(self, new_rolling_proximity_objects):
        '''Daily tracing key on ne peut pas revevoir des objet seulement des key'''
        '''Update the state'''
        self.updateState()
        print('hI FROM CONTACT SAVE')
        
        keys = new_rolling_proximity_objects['key']
        ttl = new_rolling_proximity_objects['ttl']
        for key in keys:

            if ttl == self.__time_interval_number and key not in self.__generated_rolling_proxy_table and key not in list(map(lambda x:x[1],self.__contact_rolling_proxy_table)):
                self.__contact_rolling_proxy_table.append((self.__day_number, key))

    
    #declarer cas self cas positive
    def testedPositive(self, server_url = 'http://localhost:9000/positive' ):
        '''Update the state, then get the current rolling proximity key'''
        #self.updateState()
        
        if not self.__get_infected:
            self.__get_infected = True
            
            self.__infetion_server = server_url

            diag_keys_list =[]
            for daily_tracing_key in self.__daily_tracing_key_queue:
                #day_num = daily_tracy_key.getDayNum()
                
                if daily_tracing_key is not None:
                    diag_bytes = daily_tracing_key.getKeyValue()

                    diag_64 = base64.b64encode(diag_bytes).decode('ascii')

                    diag_keys_list.append(
                        { 'day_num':self.__day_number, 'diag_key': diag_64 } 
                    )
                diagnosis_keys = {
                    'data': diag_keys_list
                    }
            
            response = requests.post(self.__infetion_server, data = json.dumps(diagnosis_keys)) 
            
            if response.status_code == 200:
                print('Success')
                print(response.text)
                
                
    def getInfectedDiagnosticKeys(self,  server_url = 'http://localhost:9000/positive/all' ):
    
        self.__infetion_server = server_url
        response = requests.get(self.__infetion_server) 
        if response.status_code == 200:
            #it return some string keys so lets just chnage it for now at least
            self.__infected_diagnostic_keys_dict = {}
            derived_diagnistic_keys = response.json()
            local_dailly_tracing_keys = list(map(lambda x:x.getKeyValue(), filter(lambda x:x!=None, self.__daily_tracing_key_queue) )) 
            #print('dervied', derived_diagnistic_keys)
            for diagnostic_key in  derived_diagnistic_keys:
                diagnostic_key_bin = base64.b64decode(diagnostic_key) 
                if diagnostic_key_bin in local_dailly_tracing_keys  and self.__get_infected==True:
                    print(self.getOwnerName(), ' Is Already infected self check from Cloud')
                    
                elif diagnostic_key_bin in local_dailly_tracing_keys and self.__get_infected==False:
                    print(self.getOwnerName(), ' Is not infected but his tracing still in Cloud as infected')
                    #later we will see if important to remove it from Cloud
                    
                elif diagnostic_key_bin not in local_dailly_tracing_keys and self.__get_infected==True:
                    self.testedPositive()
                    
                if diagnostic_key_bin not in local_dailly_tracing_keys :
                    #print('bing here ', diagnostic_key_bin)
                    if self.__day_number in self.__infected_diagnostic_keys_dict.keys() :
                        self.__infected_diagnostic_keys_dict[self.__day_number].append(diagnostic_key, )
                    else :
                        self.__infected_diagnostic_keys_dict[self.__day_number] = [diagnostic_key]
                        
            print('Success load diagnostic dailly keys')
            print(self.__infected_diagnostic_keys_dict)
            

    def isContactOccurred(self):
        '''Possitive daily tracing key list'''
        '''Here we give final situation of contact we give situation but Im not sure if we should give also the date '''
        
        '''Update the state, then get the current rolling proximity key'''
        self.updateState()
        self.getInfectedDiagnosticKeys()
        #print('dailies before insert into function', self.__infected_diagnostic_keys_dict)   
        
        self.all_possible_rolling_keys_from_diagnostic_keys = self.__current_daily_tracing_object.generateAllProximityRollingKey(self.__infected_diagnostic_keys_dict)
        print('generated possible rollings length ' ,len(self.all_possible_rolling_keys_from_diagnostic_keys) )
        print('All contact for ',self.getOwnerName(), ": " , self.__contact_rolling_proxy_table)
        for (interval, generated_rolling_proximity_key, derived_day ) in self.all_possible_rolling_keys_from_diagnostic_keys:
            
            for (contact_day, contacted_rolling_proximity_key) in self.__contact_rolling_proxy_table:
                if generated_rolling_proximity_key== contacted_rolling_proximity_key:
                    print( generated_rolling_proximity_key, contacted_rolling_proximity_key)
                    self.__infections_days.append((contact_day, interval))
                    
        if len(self.__infections_days)>0:
            for (contacted_day, interval) in self.__infections_days :
                contact_date =  date.fromtimestamp(time.time()- contacted_day )
                print(colored('Alert ‼', 'red'), self.owner_name, "Has One contact Infected Persone at day : ", contact_date, 'at hour ', int(interval*10/60), "H")
                print(colored(self.owner_name+' Get a test if possible or self quarantine and and visit the hospital when facing Complications with infection', 'red'))
                #print(self.owner_name, "Has One contact infected at day : ", num_day)
        elif  self.__get_infected==True:
            print(colored('Double Alert 💥‼', 'red'), self.owner_name, " is Tested postive His infected")
            print(colored(self.owner_name+' Self qurantine and visit the hospital when facing Complications with infection', 'red'))
        else:
            
            print(colored('Ok ✔ ', 'green'), self.owner_name, " Has No contact with infected people")
            #print(self.owner_name, " Has No contact with infected people")
        
        #print(self.all_possible_rolling_keys_from_diagnostic_keys)
    
    
    
    
    

# TESTS ON DEVICE CLASS

## TESTING DEVICES METHODES

In [8]:
d1 = TracingKeyStore("Mohamed Hassani", 111, 10)

Time interval encoded b'140'
Update state for Key Store  111 Owner  Mohamed Hassani


In [9]:

#print('Device Owner ', d1.getOwnerName())
print('Tracing Key  : ', d1.getCurrentTracingKey())
print('Length of Tracing Key', len(d1.getCurrentTracingKey()))
print('Daily Tracing Key  : ', d1.getCurrentDailyTracingKey())
print('Length of Daily Tracing Key', len(d1.getCurrentDailyTracingKey()))

p_id = d1.getCurrentRollingProximityIdentifier()
print('Rolling Proximity Identifier ', p_id.getKeyValue())
print('Length of Rolling Proximity Identifier final', len(d1.getCurrentRollingProximityIdentifier().getKeyValue()))

Tracing Key  :  mlxOIhDBywaAlYGiMJ9mNX2Ua95wiMN3
Length of Tracing Key 32
Update state for Key Store  111 Owner  Mohamed Hassani
Daily Tracing Key  :  b'\xce\x16\x91,\xd7\xdaV\x8c\xe1\xb2yY\xa6\xc4\x98j'
Update state for Key Store  111 Owner  Mohamed Hassani
Length of Daily Tracing Key 16
Update state for Key Store  111 Owner  Mohamed Hassani
Rolling Proximity Identifier  b'=I\xd8\xc4HJ7\xe4)\x00\xc63\x18\xc8\x84\xfa'
Update state for Key Store  111 Owner  Mohamed Hassani
Length of Rolling Proximity Identifier final 16


## Infections methodes

In [10]:
d1.getInfectedDiagnosticKeys()

Success load diagnostic dailly keys
{18645: ['HcxSRob8ozXlCJH12HPiXw==', 'HcxSRob8ozXlCJHnJKNljnjj', 'HcxSRob8onsdjsndjuJKJwXj']}


**declaire infection**

In [11]:
d1.testedPositive()

Success
{
  "msg": "Successfully stored", 
  "status": 200
}



**check if theer is a contact with infected pe**

In [12]:
d1.isContactOccurred()

Update state for Key Store  111 Owner  Mohamed Hassani
Mohamed Hassani  Is Already infected self check from Cloud
Success load diagnostic dailly keys
{18645: ['HcxSRob8ozXlCJH12HPiXw==', 'HcxSRob8ozXlCJHnJKNljnjj', 'HcxSRob8onsdjsndjuJKJwXj']}
generated possible rollings length  432
All contact for  Mohamed Hassani :  []
Double Alert 💥‼ Mohamed Hassani  is Tested postive His infected
Mohamed Hassani Self qurantine and visit the hospital when facing Complications with infection


# Pulic meeting 

**in this class we simulate the BLE contact rolling keys exchange in a very simple ways**

In [13]:
## HERE WE GONNA DEFINE SOME METING TO KNOW PEOPLE THAT MEET .... WE SEMULATE IN PRIMITIVE WAY THE BLE OF THE DEVICE 
### We need a table where exist all devices with an Id tha semulat BLE signal so ... 

class Meeting:
    
    def __init__(self):
        
        self.__tab_proxy_contact_tracing_people = [] # contain ids of devices in same pace ther is a BLE Link ...
        self.__tab_rolling_proximity_identifiers = [] # comtains rolling proximity identifiers generated by those devices every 10 min
    
    def getAllRollingProximityIdentifiers(self):
        return self.__tab_rolling_proximity_identifiers
    
    def addContact(self, device_id, devices_list, ttl=10): 
        '''remember that contact tracing is focusing on privacy here id device is just to semulat near distance contact between device that have Contact Tracing active'''
        if device_id not in self.__tab_proxy_contact_tracing_people:
            self.__tab_proxy_contact_tracing_people.append(device_id)
            
            for device in devices_list:
                if device.getDeviceId() == device_id:
                    device_rolling_proximity_identifier = device.getCurrentRollingProximityIdentifier().getKeyValue()
                    if device_rolling_proximity_identifier not in self.__tab_rolling_proximity_identifiers:
                        self.__tab_rolling_proximity_identifiers.append(device_rolling_proximity_identifier)
                    
            for device in devices_list:
                device.saveContact({'key':self.__tab_rolling_proximity_identifiers, 'ttl':ttl})
                
            print('HI---------------------\n', self.__tab_rolling_proximity_identifiers)

In [14]:
# TEST MEETHING

seconds_since_start_of_day_number =  datetime.datetime.now().timestamp()% (60*60*24)
ttl= int(seconds_since_start_of_day_number/(60*10)) 
        
devices = []

d2 = TracingKeyStore("Amine Reda", 222)
devices.append(d2)

d3 = TracingKeyStore("Meyiam Salhi", 333)
devices.append(d3)

meeting_1 = Meeting()

meeting_1.addContact(222, devices, ttl)
meeting_1.addContact(333, devices, ttl)


d2.getInfectedDiagnosticKeys()
d3.getInfectedDiagnosticKeys()
print(devices)

Time interval encoded b'140'
Update state for Key Store  222 Owner  Amine Reda
Time interval encoded b'140'
Update state for Key Store  333 Owner  Meyiam Salhi
Update state for Key Store  222 Owner  Amine Reda
Update state for Key Store  222 Owner  Amine Reda
hI FROM CONTACT SAVE
Update state for Key Store  333 Owner  Meyiam Salhi
hI FROM CONTACT SAVE
HI---------------------
 [b'\x1cM\xc0-\xeaN\x8a\x1d\xcar\x98\xc6lqG\xad']
Update state for Key Store  333 Owner  Meyiam Salhi
Update state for Key Store  222 Owner  Amine Reda
hI FROM CONTACT SAVE
Update state for Key Store  333 Owner  Meyiam Salhi
hI FROM CONTACT SAVE
HI---------------------
 [b'\x1cM\xc0-\xeaN\x8a\x1d\xcar\x98\xc6lqG\xad', b'\xb2\xa2Hf/\xebR\x1bl\xc5(\\\x13\xe5\xb4#']
Success load diagnostic dailly keys
{18645: ['HcxSRob8ozXlCJH12HPiXw==', 'HcxSRob8ozXlCJHnJKNljnjj', 'HcxSRob8onsdjsndjuJKJwXj', 'zhaRLNfaVozhsnlZpsSYag==']}
Success load diagnostic dailly keys
{18645: ['HcxSRob8ozXlCJH12HPiXw==', 'HcxSRob8ozXlCJHnJKNljnjj

In [15]:
ttl

140

**Contact Rolling keys circulating ins this meeting of conctact**

In [16]:
meeting_1.getAllRollingProximityIdentifiers()

[b'\x1cM\xc0-\xeaN\x8a\x1d\xcar\x98\xc6lqG\xad',
 b'\xb2\xa2Hf/\xebR\x1bl\xc5(\\\x13\xe5\xb4#']

### We test a user as positive

In [17]:
d2.testedPositive()

Success
{
  "msg": "Successfully stored", 
  "status": 200
}



In [18]:
d2.getInfectedDiagnosticKeys()

Amine Reda  Is Already infected self check from Cloud
Success load diagnostic dailly keys
{18645: ['HcxSRob8ozXlCJH12HPiXw==', 'HcxSRob8ozXlCJHnJKNljnjj', 'HcxSRob8onsdjsndjuJKJwXj', 'zhaRLNfaVozhsnlZpsSYag==']}


In [19]:
d2.isContactOccurred()

Update state for Key Store  222 Owner  Amine Reda
Amine Reda  Is Already infected self check from Cloud
Success load diagnostic dailly keys
{18645: ['HcxSRob8ozXlCJH12HPiXw==', 'HcxSRob8ozXlCJHnJKNljnjj', 'HcxSRob8onsdjsndjuJKJwXj', 'zhaRLNfaVozhsnlZpsSYag==']}
generated possible rollings length  576
All contact for  Amine Reda :  [(18645, b'\x1cM\xc0-\xeaN\x8a\x1d\xcar\x98\xc6lqG\xad'), (18645, b'\xb2\xa2Hf/\xebR\x1bl\xc5(\\\x13\xe5\xb4#')]
Double Alert 💥‼ Amine Reda  is Tested postive His infected
Amine Reda Self qurantine and visit the hospital when facing Complications with infection


In [20]:
d3.isContactOccurred()

Update state for Key Store  333 Owner  Meyiam Salhi
Success load diagnostic dailly keys
{18645: ['HcxSRob8ozXlCJH12HPiXw==', 'HcxSRob8ozXlCJHnJKNljnjj', 'HcxSRob8onsdjsndjuJKJwXj', 'zhaRLNfaVozhsnlZpsSYag==', 'Dyn2bJLPQ9/CQ2IzH+8RNg==']}
generated possible rollings length  720
All contact for  Meyiam Salhi :  [(18645, b'\x1cM\xc0-\xeaN\x8a\x1d\xcar\x98\xc6lqG\xad'), (18645, b'\xb2\xa2Hf/\xebR\x1bl\xc5(\\\x13\xe5\xb4#')]
b'\x1cM\xc0-\xeaN\x8a\x1d\xcar\x98\xc6lqG\xad' b'\x1cM\xc0-\xeaN\x8a\x1d\xcar\x98\xc6lqG\xad'
Alert ‼ Meyiam Salhi Has One contact Infected Persone at day :  2021-01-18 at hour  23 H
Meyiam Salhi Get a test if possible or self quarantine and and visit the hospital when facing Complications with infection


**Like we see we have a contact with an infected user**

**Some useless logs**

In [21]:
d3.getCurrentRollingProximityIdentifier()
meeting_1.addContact(333, devices)
meeting_1.getAllRollingProximityIdentifiers()

Update state for Key Store  333 Owner  Meyiam Salhi


[b'\x1cM\xc0-\xeaN\x8a\x1d\xcar\x98\xc6lqG\xad',
 b'\xb2\xa2Hf/\xebR\x1bl\xc5(\\\x13\xe5\xb4#']